In [12]:
import pandas as pd
import numpy as np

def calculate_eigenvector(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_eigenvalue = np.max(eigenvalues)
    max_eigenvector = eigenvectors[:, np.argmax(eigenvalues)]
    normalized_vector = max_eigenvector / np.sum(max_eigenvector)
    return max_eigenvalue, normalized_vector.real

def calculate_consistency(matrix, max_eigenvalue):
    n = matrix.shape[0]
    ci = (max_eigenvalue - n) / (n - 1)
    ri_values = {1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}
    ri = ri_values.get(n, 1.49)
    cr = ci / ri if ri != 0 else 0
    return ci, cr, ri

def geometric_mean(matrices):
    product_matrix = np.ones_like(matrices[0])
    for matrix in matrices:
        product_matrix *= matrix
    return np.power(product_matrix, 1/len(matrices))

# Read the CSV file
file_path = './data/physical_construction.csv.csv'
data = pd.read_csv(file_path)

# Parse the CSV into individual matrices
engineer_matrices = {}
for i in range(0, data.shape[1], 3):
    sub_criteria = data.iloc[:, i:i+3].values
    matrix = np.eye(3)
    matrix[0, 1] = sub_criteria[0, 0]
    matrix[0, 2] = sub_criteria[0, 1]
    matrix[1, 2] = sub_criteria[0, 2]
    matrix[1, 0] = 1 / matrix[0, 1]
    matrix[2, 0] = 1 / matrix[0, 2]
    matrix[2, 1] = 1 / matrix[1, 2]
    engineer_matrices[f'Engineer {i//3 + 1}'] = matrix

consistent_matrices = []
results = []

# Process each matrix
for engineer, matrix in engineer_matrices.items():
    max_eigenvalue, weights = calculate_eigenvector(matrix)
    ci, cr, ri = calculate_consistency(matrix, max_eigenvalue)
    if cr <= 0.1:
        consistent_matrices.append(matrix)
    results.append({
        'engineer': engineer,
        'matrix': matrix,
        'weights': weights,
        'max_eigenvalue': max_eigenvalue,
        'ci': ci,
        'cr': cr,
        'ri': ri
    })

# Aggregate the matrices
if consistent_matrices:
    aggregate_matrix = geometric_mean(consistent_matrices)
    max_eigenvalue, weights = calculate_eigenvector(aggregate_matrix)
    ci, cr, ri = calculate_consistency(aggregate_matrix, max_eigenvalue)
    aggregate_result = {
        'aggregate_matrix': aggregate_matrix,
        'weights': weights,
        'max_eigenvalue': max_eigenvalue,
        'ci': ci,
        'cr': cr,
        'ri': ri
    }

# Print results
for result in results:
    print(f"Engineer: {result['engineer']}")
    print("Pairwise Comparison Matrix:\n", result['matrix'])
    print("Weights:\n", result['weights'])
    print("Max Eigenvalue:", result['max_eigenvalue'])
    print("Consistency Index (CI):", result['ci'])
    print("Consistency Ratio (CR):", result['cr'])
    print("Random Index (RI):", result['ri'])
    print("\n")

if consistent_matrices:
    print("Aggregate Pairwise Comparison Matrix:\n", aggregate_result['aggregate_matrix'])
    print("Aggregate Weights:\n", aggregate_result['weights'])
    print("Aggregate Max Eigenvalue:", aggregate_result['max_eigenvalue'])
    print("Aggregate Consistency Index (CI):", aggregate_result['ci'])
    print("Aggregate Consistency Ratio (CR):", aggregate_result['cr'])
    print("Aggregate Random Index (RI):", aggregate_result['ri'])
else:
    print("No consistent matrices found to aggregate.")


Engineer: Engineer 1
Pairwise Comparison Matrix:
 [[1.         1.         3.        ]
 [1.         1.         1.        ]
 [0.33333333 1.         1.        ]]
Weights:
 [0.45995809 0.31891713 0.22112479]
Max Eigenvalue: (3.1356108446580446+0j)
Consistency Index (CI): (0.06780542232902231+0j)
Consistency Ratio (CR): (0.11690590056727986+0j)
Random Index (RI): 0.58


No consistent matrices found to aggregate.
